# Old code copied from GH script (Y:\ituran\Behnisch\Philly_Views\ExternalViewAnalysis_Philly_190820.gh)

In [ ]:
# INPUTS: 
# dirPath = path where all is saved
# name = name of simulation test (names all files)
# viewType = the ray projection type
# vects = the ray vectors 
# pts = tree with points in grid
# exteriorMaterials = the materials assigned in DIVA that are considered outside materials


##########################################
# Calculate number of objects, materials, and distances
##########################################
# .dat output data includes origin xyz, material, distance, destination surface name 

# Read in the list of exterior materials
# Clear empty lines at end of the list, turn into dictionary 
materialList = list(filter(None, exteriorMaterials))

# Read the results .dat file, spliting by tab delimiter
resultsAll = []
with open(dirPath + '\\' + name + '.dat', 'r') as f: 
    for l in f: 
        resultsAll.append(l.strip().split('\t'))

# Convert numbers from string to float 
for i in resultsAll: 
    i[0] = float(i[0])
    i[1] = float(i[1])
    i[2] = float(i[2])
    i[4] = float(i[4])

# Get the list of unique origin x,y,z from resultsAll
resultsOrigin = [item[0:3] for item in resultsAll]
resultsOrigin.sort()
originsUnique = list(k for k,_ in itertools.groupby(resultsOrigin))

# Get list of all materials hit from resultsAll
resultsMat = [item[3] for item in resultsAll]
resultsMat.sort()
materialsAll = list(k for k,_ in itertools.groupby(resultsMat))

# Make initial datatree
resultsTree = DataTree[System.Object]()

# Read the results file
#keys = ['point', 'totalRays', 'totalMaterials', 'totalObjects', 'distanceRange']
resultsDict = []

for i in range(len(originsUnique)): 
    ptResults = []
    ptObjects = set([])    
    totalRayCount = 0
    
    #extDistAll = []
    matCountDict = {x:0 for x in materialList}
    matCountDict_allMat = {x:0 for x in materialsAll}
    
    for j in resultsAll:
        # Check if line is for the unique point
        if j[0:3] == originsUnique[i]:
            totalRayCount += 1
            
            # Count rays to all materials that are present in the scene
            matCountDict_allMat[j[3]] += 1
            
            """
            # Check if ray is in external materials list; if so, add to count
            if j[3] in matCountDict.keys(): 
                matCountDict[j[3]] += 1 
                ptObjects.add(j[5])
                
                
                # Check min/max distance
                if j[3] != 'sky_mat':
                    extDistAll.append(j[4])
            else: 
                continue
            """
            
        else: 
            continue 
    
    print(originsUnique[i])
    #print(matCountDict)
    print(matCountDict_allMat)
    
    # Get total number of UNIQUE objects
    #sumExtObjs = len(ptObjects)
    
    """
    # Get min and max exterior distance viewed from origin point (not including sky)
    if len(extDistAll) > 0: 
        ptDistMax = max(x for x in extDistAll)
        ptDistMin = min(x for x in extDistAll)
        viwDistRange = ptDistMax-ptDistMin
        #viwDistRange = sum(extDistAll)/float(len(extDistAll))
    else: 
        viwDistRange = 0
    
    print(viwDistRange)
    """
    
    # Create a new branch in the tree for the point
    path = GH_Path(i)
    
    # Add results to datatree for GH use
    resultsTree.Add(rc.Geometry.Point3d(originsUnique[i][0],originsUnique[i][1], originsUnique[i][2]), path)
    resultsTree.Add(totalRayCount, path)
    #resultsTree.Add(sumExtObjs, path)
    #resultsTree.Add(viwDistRange, path)

    # Add each material ray count to tree
    for key in materialsAll: 
        resultsTree.Add(matCountDict_allMat[key], path)
        
    """
    # Add each material ray count to tree
    for key in materialList: 
        resultsTree.Add(matCountDict[key], path)
    """
    
# Make legend for the resultsTree
resultsTreeLegend = DataTree[System.Object]()
resultsTreeLegend.Add('origin point') 
resultsTreeLegend.Add('total ray count') 
#resultsTreeLegend.Add('total exterior objects') 
#resultsTreeLegend.Add('total exterior distance') 

for key in materialsAll: 
    resultsTreeLegend.Add(key) 

In [1]:
import sys
import os 
from subprocess import Popen
import glob
import fileinput
import shutil 
import csv
import math

from pathlib import Path
import numpy
import pandas as pd

# Read results file into dataframe

In [15]:
csvfile = r'C:\Users\ituran\NYCstudy\testFolder\1000007\out_1000007_6.dat'


# Read csv file in to df
data = pd.read_csv(csvfile, sep="\t", header=None, index_col=False)
data = data.drop(data.columns[-1], axis=1)
data.columns = ["x", "y", "z", "mat", "dist", "destination"]

#print(data.head())
#print(data.shape)

          x         y        z                     mat          dist  \
0 -18604.39 -4605.049  66.8856  GenericInteriorWall_50  1.899185e+00   
1 -18604.39 -4605.049  66.8856   Glazing_SinglePane_88  6.983244e+00   
2 -18604.39 -4605.049  66.8856   Glazing_SinglePane_88  1.000000e+10   
3 -18604.39 -4605.049  66.8856  GenericInteriorWall_50  2.662185e+00   
4 -18604.39 -4605.049  66.8856       GenericCeiling_70  1.971831e+01   

                 destination  
0  GenericInteriorWall_50.46  
1   Glazing_SinglePane_88.59  
2   Glazing_SinglePane_88.58  
3  GenericInteriorWall_50.47  
4        GenericCeiling_70.1  
(357155, 6)


In [20]:
# Create a df with only the unique origins 
dataOrigin = data.filter(['x', 'y', 'z'], axis=1)
dataOrigin = dataOrigin.drop_duplicates()

#print(dataOrigin.head())
#print(dataOrigin.shape)

#Check that all floors are represented
print("z value of all floors:")
print(dataOrigin["z"].unique())

             x         y        z
0    -18604.39 -4605.049  66.8856
305  -18603.22 -4610.276  66.8856
610  -18601.89 -4616.233  66.8856
915  -18600.56 -4622.190  66.8856
1220 -18599.22 -4628.147  66.8856
(1171, 3)


In [21]:
# Create a df of all the material types that are hit
dataMat = data.filter(['mat'], axis=1)
dataMat = dataMat.drop_duplicates()

#print(dataMat.head())
#print(dataMat.shape)

                        mat
0    GenericInteriorWall_50
1     Glazing_SinglePane_88
4         GenericCeiling_70
114         GenericFloor_20
(4, 1)
